In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision
from torchvision import models
import torch.optim as optim
import pandas as pd
import numpy as np
import cv2
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt
from scipy.io import arff
%matplotlib inline
import kornia
from sklearn.pipeline import Pipeline
from collections import defaultdict
from sklearn.preprocessing import *

In [3]:
root_dir = '/scratch/prathyuakundi/aicrowd/retina/'

train_data_path = root_dir+"train.csv"
train_data_df = pd.read_csv(train_data_path, header=None)

In [4]:
train_data_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,1,75,63,60,55,48,35,13.195493,4.396967,0.104070,0.000000,0.000000,0.000000,0.000000,0.000000,0.513092,0.123966,0,1
1,1,1,79,76,74,72,69,50,61.559348,28.959444,12.778104,2.045287,0.038016,0.000000,0.000000,0.000000,0.527993,0.101884,0,1
2,1,1,41,41,40,40,38,35,6.090116,0.834492,0.027460,0.000000,0.000000,0.000000,0.000000,0.000000,0.506881,0.091535,1,0
3,1,1,17,16,16,14,12,9,75.438535,20.352500,5.237412,0.206817,0.003884,0.000971,0.000971,0.000971,0.544614,0.089329,1,1
4,1,1,63,63,63,59,57,48,13.558211,5.366467,0.604079,0.051511,0.000000,0.000000,0.000000,0.000000,0.552941,0.112387,0,1


In [5]:
train_data_df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000,920.000000
mean,0.995652,0.914130,38.316304,36.791304,35.010870,32.181522,28.673913,21.064130,63.698175,22.905469,8.519814,1.798095,0.551911,0.194753,0.075390,0.031751,0.523033,0.108823,0.331522,0.523913
std,0.065830,0.280324,25.179680,23.678384,22.433359,20.835192,19.274834,14.887508,58.435639,21.462560,11.294791,4.009692,2.598086,1.051588,0.371368,0.152491,0.028224,0.018132,0.471016,0.499699
min,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.349274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.367762,0.057906,0.000000,0.000000
25%,1.000000,1.000000,16.000000,16.000000,15.000000,13.750000,11.000000,8.000000,22.367841,7.930734,1.235325,0.079222,0.000000,0.000000,0.000000,0.000000,0.502463,0.096086,0.000000,0.000000
50%,1.000000,1.000000,36.000000,35.000000,32.000000,30.000000,25.500000,18.000000,44.233267,17.041567,4.433600,0.487339,0.023293,0.001280,0.000000,0.000000,0.523561,0.107282,0.000000,1.000000
75%,1.000000,1.000000,55.000000,53.000000,51.000000,47.250000,43.000000,32.000000,86.799120,30.451345,11.424670,1.863160,0.193138,0.037618,0.004830,0.003825,0.544011,0.120440,1.000000,1.000000
max,1.000000,1.000000,147.000000,132.000000,113.000000,94.000000,86.000000,77.000000,403.939108,167.131427,106.070092,59.766121,51.423208,20.098605,5.937799,2.171842,0.592217,0.219199,1.000000,1.000000


In [6]:
# del train_data_df['0'],train_data_df['1'],train_data_df['18']

In [7]:
train_data_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,1,75,63,60,55,48,35,13.195493,4.396967,0.104070,0.000000,0.000000,0.000000,0.000000,0.000000,0.513092,0.123966,0,1
1,1,1,79,76,74,72,69,50,61.559348,28.959444,12.778104,2.045287,0.038016,0.000000,0.000000,0.000000,0.527993,0.101884,0,1
2,1,1,41,41,40,40,38,35,6.090116,0.834492,0.027460,0.000000,0.000000,0.000000,0.000000,0.000000,0.506881,0.091535,1,0
3,1,1,17,16,16,14,12,9,75.438535,20.352500,5.237412,0.206817,0.003884,0.000971,0.000971,0.000971,0.544614,0.089329,1,1
4,1,1,63,63,63,59,57,48,13.558211,5.366467,0.604079,0.051511,0.000000,0.000000,0.000000,0.000000,0.552941,0.112387,0,1


In [8]:
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [9]:
train_data = train_data_df.to_numpy()

In [10]:
scaler = preprocessing.StandardScaler()#QuantileTransformer(random_state=42)#.PowerTransformer()#StandardScaler()

X_train, X_val= train_test_split(train_data, test_size=0.3, random_state=42)

X_train,y_train = X_train[:,:-1],X_train[:,-1]
X_val,y_val = X_val[:,:-1],X_val[:,-1]

scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
X_train_ = scaler.transform(X_train)
X_val_ = scaler.transform(X_val)

In [46]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                 hidden_layer_sizes=(5, 2), random_state=1)

In [47]:
model.fit(X_train_, y_train)

/home/prathyuakundi/miniconda3/envs/tenv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [48]:
predicted_MLP = model.predict(X_val_)

In [49]:
accuracy_score(y_val, predicted_MLP)

0.7137681159420289

In [50]:
predicted_MLP = model.predict(X_train_)

In [51]:
accuracy_score(y_train, predicted_MLP)

0.7608695652173914

In [26]:
final_test_path = root_dir+"test.csv"
final_test = pd.read_csv(final_test_path, header=None)

In [27]:
final_test_ = scaler.transform(final_test)

In [28]:
predicted_MLP = model.predict(final_test_)

In [30]:
len(predicted_MLP)

230

In [31]:
submission = pd.DataFrame(predicted_MLP)
submission.to_csv('submission.csv',header=['label'],index=False)

In [20]:
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.ensemble import *
from sklearn import tree
seed = 7

kfold = model_selection.KFold(n_splits=10, random_state=seed)

In [21]:
estimators = []
model1 = RandomForestClassifier()
estimators.append(('Random Forest', model1))
model2 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=1, random_state=0)
estimators.append(('GBC', model2))
model3 = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=200)
estimators.append(('ADB', model3))
model4 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
estimators.append(('MLP', model4))

In [22]:
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X_train, y_train, cv=kfold)

In [23]:
print(results.mean())

0.6571141975308642


In [24]:
results

array([0.64197531, 0.69135802, 0.69135802, 0.67901235, 0.60493827,
       0.6625    , 0.5875    , 0.6       , 0.75      , 0.6625    ])